## You need start ollama server first in cmd run (ollama serve) 

In [6]:
import requests

response = requests.get("http://localhost:11434/v1/models")
response.raise_for_status()
data = response.json()
models = data['data']
models_id = [model['id'] for model in models]
models_id

['deepseek-r1:14b', 'deepseek-r1:32b', 'deepseek-r1:8b']

In [10]:
import ipywidgets as widgets
sw = widgets.Select(
    options=models_id,
    value=models_id[0],
    rows=10,
    description='please select model:',
    disabled=False
)

In [11]:
import ipywidgets as widgets
from IPython.display import display
output = widgets.Output()

@output.capture(clear_output=False,wait=True) # based on https://github.com/jupyter-widgets/ipywidgets/issues/1846 and https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html
def sayHello(b):
    print("selected:", sw.value)

run_button = widgets.Button(
    description = 'continue'
)
print("Hello, please select model")
run_button.on_click(sayHello)
display(sw)
display(run_button)
output

Hello, please select model


Select(description='please select model:', options=('deepseek-r1:14b', 'deepseek-r1:32b', 'deepseek-r1:8b'), r…

Button(description='continue', style=ButtonStyle())

Output()

In [12]:
# !pip install openai aiohttp nest_asyncio

import json
from IPython.display import Markdown, display
import textwrap
import asyncio
import openai
import aiohttp
import nest_asyncio

nest_asyncio.apply()
openai.api_key = 'lm-studio'

code_highlight = ", the code in response will be like this ```{program_language_name}"

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

async def fetch_completion(my_msg, code_highlights = False):
    async with aiohttp.ClientSession() as session:
        async with session.post(
            'http://localhost:11434/v1/chat/completions',
            headers={
                # 'Authorization': f'Bearer {openai.api_key}',
                'Content-Type': 'application/json'
            },
            json={
                'model': str(sw.value),
                "messages": [
                  { "role": "system", "content": "if code is in response add the code like this ```{program_language_name}" },
                  { "role": "user", "content": my_msg if not code_highlights else my_msg + code_highlight }
                ],
                'temperature': 0.7,
                "max_tokens": -1, # -1 for unlimited
                "stream": True
            }
        ) as response:
            content = ""
            display_handle = display(to_markdown(content), display_id=True)
            async for line in response.content:
                line = line.decode('utf-8').strip()
                if line.startswith("data: "):
                    data = line[len("data: "):]
                    if data != '[DONE]':
                        json_data = json.loads(data)
                        delta = json_data['choices'][0]['delta']
                        if 'content' in delta:
                            content += delta['content']
                            display_handle.update(to_markdown(content))
            return content

# prompt = "Once upon a time"
# full_ans = await fetch_completion(prompt)

In [13]:
prompt = "write in max 100 words python code that get url data"
full_ans = await fetch_completion(prompt, code_highlights=True)
# print(full_ans)

> <think>
> Okay, I need to write a Python function called scrape_url_data. The goal is to extract data from a URL when given as input. Hmm, first, I should figure out how URLs are structured. URLs typically have different parts like protocol (http, https), domain, path, query parameters, and fragment.
> 
> I remember that the urllib library in Python has modules that can help parse URLs. Oh right, there's urlparse from urllib.parse. So I'll probably use that to break down the URL into its components.
> 
> Let me outline what each part means: scheme is http or https, netloc is the domain, path is the part after the domain, query is the parameters after the '?', fragment is anything after '#'. The function should return these parts as a dictionary with keys like 'scheme', 'netloc', 'path', 'query', and 'fragment'.
> 
> I'll start by importing urlparse from urllib.parse. Then, inside the function, I'll take the URL input. Next, use urlparse(url) to get the components. Wait, but sometimes URLs might not be parsed correctly if they're malformed. But since this is a basic example, I can assume the URL is valid.
> 
> I should handle cases where some parts are empty or None. For instance, the scheme and netloc can't be empty. So maybe include checks for that, but perhaps it's beyond the scope of this simple function.
> 
> Now, construct a dictionary with each component. Make sure to convert the path into a string in case it's already a bytes object. Also, handle any potential encoding issues, but again, maybe not necessary here.
> 
> Testing an example: if the URL is 'https://www.example.com/path?query=123#fragment', the function should return scheme='https', netloc='www.example.com', path='/path', query='query=123', fragment='fragment'.
> 
> I think that's it. Now, write the code step by step, making sure each part is addressed.
> </think>
> 
> To extract URL data using Python, we can utilize the `urlparse` function from the `urllib.parse` module. Here's a concise example:
> 
> ```python
> from urllib.parse import urlparse
> 
> def scrape_url_data(url):
>     url_components = urlparse(url)
>     return {
>         'scheme': url_components.scheme,
>         'netloc': url_components.netloc,
>         'path': str(url_components.path),
>         'query': url_components.query,
>         'fragment': url_components.fragment
>     }
> ```
> 
> This function breaks down a given URL into its components and returns them as a dictionary. Each key corresponds to a specific part of the URL, making it easy to analyze or process individual elements.